# Для начала главное
Вот Вам именинный котя, милый и залипательный :3

![alt text](https://bit.ua/wp-content/uploads/2013/12/giphy-4.gif)


А теперь уже можно и к домашкам там всяким переходить)

# Отчёт

Мы попробовали 2 способа совместить пространства эмбеддингов для разных языков: обучив регрессор и через разложение матриц. Через SVD получается намного лучше, а учитывая небольшую разницу между точностью в первом варианте и точностью в первых пяти, можно сказать, что большинство правильных переводов предлагаются первым вариантом.

Мы даже сделали на основе SVD переводчик, правда, он пословный и глупенький.

In [0]:
print('''\tRegression\t\tSVD
TOP-1:\t{}\t{}
TOP-5:\t{}\t{}
'''.format(map_precision_1, map_precision_5, svd_precision_1, svd_precision_5))

	Regression		SVD
TOP-1:	0.6356589147286822	0.813953488372093
TOP-5:	0.6537467700258398	0.8242894056847545



#Homework: Multilingual Embedding-based Machine Translation

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully). 

For our system we choose two kindred Slavic languages: Ukrainian and Russian. 

##Data

In [0]:

import gensim
import numpy as np
from gensim.models import KeyedVectors

Download embeddings here:
* [cc.uk.300.vec.zip](https://yadi.sk/d/9CAeNsJiInoyUA)
* [cc.ru.300.vec.zip](https://yadi.sk/d/3yG0-M4M8fypeQ)

In [0]:
!wget  https://www.dropbox.com/s/mqmilqwlo8o81vi/cc.uk.300.vec.zip
!unzip cc.uk.300.vec.zip

--2019-05-16 15:26:54--  https://drive.google.com/file/d/1Hn7XcwT5Sq72FlMiwK0yWj9yH8gVryON/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.141.113, 74.125.141.100, 74.125.141.102, ...
Connecting to drive.google.com (drive.google.com)|74.125.141.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=sharing’

view?usp=sharing        [ <=>                ] 119.82K  --.-KB/s    in 0.009s  

2019-05-16 15:26:54 (12.8 MB/s) - ‘view?usp=sharing’ saved [122695]



In [0]:
!wget  https://www.dropbox.com/s/nvhpu2dfinbbtqq/cc.ru.300.vec.zip
!unzip cc.ru.300.vec.zip

--2019-03-03 15:40:31--  https://www.dropbox.com/s/nvhpu2dfinbbtqq/cc.ru.300.vec.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nvhpu2dfinbbtqq/cc.ru.300.vec.zip [following]
--2019-03-03 15:40:31--  https://www.dropbox.com/s/raw/nvhpu2dfinbbtqq/cc.ru.300.vec.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5ec767b544ac2979732a627e0f.dl.dropboxusercontent.com/cd/0/inline/AcY8sCS9S2dA7kgCvtBHP36FTwVBx-sqmmD93M6x40l3dxhdM6WHQFRx15vIoiTilsXwcDt00LdCCV41TyUgIhmbs9G4thK5Ugh2w7QqFKlRQnUsugIAn3T-cl2X9_XM9i4/file# [following]
--2019-03-03 15:40:31--  https://uc5ec767b544ac2979732a627e0f.dl.dropboxusercontent.com/cd/0/inline/AcY8sCS9S2dA7kgCvtBHP36FTwVBx-sqmmD93M6x40l3dxhdM6WHQFRx15vIoiTilsXwcDt00LdCCV41TyUgIhmbs9G4thK5Ugh

Load embeddings for ukrainian and russian.

In [0]:
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [0]:
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [0]:
!wget https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
!wget https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
!wget https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/fairy_tale.txt  

--2019-03-03 15:45:57--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59351 (58K) [text/plain]
Saving to: ‘ukr_rus.train.txt.1’

ukr_rus.train.txt.1 100%[===================>]  57.96K  --.-KB/s    in 0.02s   

2019-03-03 15:45:57 (2.76 MB/s) - ‘ukr_rus.train.txt.1’ saved [59351/59351]

--2019-03-03 15:45:59--  https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week01_embeddings/ukr_rus.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting r

In [0]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [0]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [0]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

In Greek mythology, Procrustes or "the stretcher" was a rogue smith and bandit from Attica who attacked people by stretching them or cutting off their legs, so as to force them to fit the size of an iron bed. We make same bad things with source embedding space. Our Procrustean bed is target embedding space.

![embedding_mapping.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/embedding_mapping.png)

![procrustes.png](https://github.com/yandexdataschool/nlp_course/raw/master/resources/procrustes.png)

But wait...$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [0]:
from sklearn.linear_model import LinearRegression

mapping = LinearRegression(fit_intercept=False)

In [0]:
mapping.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None,
         normalize=False)

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [0]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('апрель', 0.8541285991668701),
 ('июнь', 0.8411202430725098),
 ('март', 0.839699387550354),
 ('сентябрь', 0.835986852645874),
 ('февраль', 0.8329296708106995),
 ('октябрь', 0.8311846256256104),
 ('ноябрь', 0.8278923034667969),
 ('июль', 0.8234529495239258),
 ('август', 0.8120501041412354),
 ('декабрь', 0.8039004802703857)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [0]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        variants = ru_emb.most_similar(mapped_vectors[i].reshape(1, -1), topn=topn) # берём k предсказаний
        #print(variants)
        ru_variants = [variant[0] for variant in variants] # собираем список вариантов
        #print(ru_variants)
        if ru in ru_variants:
          num_matches += 1
    precision_val = num_matches / len(pairs)
    return precision_val

In [0]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## Making it better (orthogonal Procrustean problem)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [0]:
from numpy.linalg import svd

def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    U, S, V = svd(np.matmul(X_train.T, Y_train))
    return np.matmul(U, V)

In [0]:
W = learn_transform(X_train, Y_train)

In [0]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('апрель', 0.8237907290458679),
 ('сентябрь', 0.8049713373184204),
 ('март', 0.802565336227417),
 ('июнь', 0.8021842241287231),
 ('октябрь', 0.8001736402511597),
 ('ноябрь', 0.793448269367218),
 ('февраль', 0.7914119958877563),
 ('июль', 0.7908108234405518),
 ('август', 0.789101779460907),
 ('декабрь', 0.7686372995376587)]

In [0]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


## UK-RU Translator

Now we are ready to make simple word-based translator: for earch word in source language in shared embedding space we find the nearest in target language.


In [0]:
with open("fairy_tale.txt", "r") as inpf:
    uk_sentences = [line.rstrip().lower() for line in inpf]

In [0]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translate = []
    tokens = tokenizer.tokenize(sentence)
    for token in tokens:
      if token in uk_emb.vocab:
        uk = np.matmul(uk_emb.get_vector(token), W) # матрицу W уже обучили
        #print(uk)
        ru = ru_emb.most_similar([uk], topn=1)[0][0] # берём слово из кортежа наиболее похожего вектора
        #print(ru)
        translate.append(ru)
      else: # если такого токена нет в словаре, то оставим его так, но с пометочкой
        translate.append('_'+token+'_')
    return ' '.join(translate)

In [0]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
for sentence in uk_sentences:
    print("ukr: {}\nrus: {}\n".format(sentence, translate(sentence)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ukr: лисичка - сестричка і вовк - панібрат
rus: лисичка – сестричка и волк – _панібрат_

ukr: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
rus: как была себе лисичка , че и пошла раз к _однії_ бабы _добувать_ огня ; вошла во избу че и говорит : " хороший день тебе , бабушку !

ukr: дай мені огня " .
rus: дай мне огня " .

ukr: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
rus: а бабка только что вынула со печи _пирожок_ со маком , сладкий , че и согнула , чтобы он _прохолов_ ; а лисичка ой и _підгледала_ , че токмо что бабка качнулась во печь , чтобы _достать_ огня , то лисичка сейчас _ухватила_ _пирожок_ че и деру со хаты , че , пробежать , весь мак со его _

Not so bad, right? We can easily improve translation using language model and not one but several nearest neighbours in shared embedding space. But next time.

## Оценка

In [0]:
map_precision_1 = precision(uk_ru_test, mapping.predict(X_test), 1)
map_precision_5 = precision(uk_ru_test, mapping.predict(X_test), 5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
svd_precision_1 = precision(uk_ru_test, np.matmul(X_test, W), 1)
svd_precision_5 = precision(uk_ru_test, np.matmul(X_test, W), 5)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
print('''\tRegression\t\tSVD
TOP-1:\t{}\t{}
TOP-5:\t{}\t{}
'''.format(map_precision_1, map_precision_5, svd_precision_1, svd_precision_5))

	Regression		SVD
TOP-1:	0.6356589147286822	0.813953488372093
TOP-5:	0.6537467700258398	0.8242894056847545

